In [1]:
from dataset import PrototypicalCUB
from trainer import PrototypicalTrainer
from utils import PrototypicalBatchSampler

from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch

import os
import numpy as np

In [2]:
# configs
CLASSES_PER_IT_TR = 5
CLASSES_PER_IT_VAL = 5
SAMPLES_PER_CLASS = 1
ITERATIONS = 100
NUM_SUPPORT_TRAINING = 1
NUM_QUERY_TRAINING = 5
NUM_SUPPORT_VAL = 1
NUM_QUERY_VAL = 5
LEARNING_RATE = 0.001
LR_GAMMA = 0.5
LR_STEP = 20
EPOCHS = 10

In [3]:
# checking for gpu availability
torch.cuda.is_available()

True

In [4]:
# loading the googlenet model
model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)

Using cache found in C:\Users\ani_1/.cache\torch\hub\pytorch_vision_v0.10.0


In [5]:
# creating the path
data_path = os.path.join(os.getcwd(), 'CUB_200_2011')
print(data_path)

C:\Users\ani_1\PycharmProjects\Thesis\CUB_200_2011


In [6]:
model_path = os.path.join(os.getcwd(), 'prototypicalModels')

### Basic Preprocessing steps

In [7]:
train_transforms = transforms.Compose([
        transforms.RandomResizedCrop(160),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [8]:
val_transforms = transforms.Compose([
        transforms.Resize(192),
        transforms.CenterCrop(160),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [9]:
torch.manual_seed(7)
torch.cuda.manual_seed(7)

In [10]:
train_dataset = PrototypicalCUB(path=data_path, mode='train', transform=train_transforms)
val_dataset = PrototypicalCUB(path=data_path, mode='val', transform=val_transforms)
test_dataset = PrototypicalCUB(path=data_path, mode='test', transform=val_transforms)

In [11]:
train_sampler = PrototypicalBatchSampler(labels=train_dataset.class_ids,
                                         data=train_dataset.data_id,
                                         classes_per_it=CLASSES_PER_IT_TR, 
                                         num_samples= NUM_SUPPORT_TRAINING+NUM_QUERY_TRAINING,
                                         iterations= ITERATIONS)

In [12]:
val_sampler = PrototypicalBatchSampler(labels=val_dataset.class_ids,
                                       data=val_dataset.data_id,
                                      classes_per_it=CLASSES_PER_IT_VAL,
                                      num_samples= NUM_QUERY_VAL+NUM_SUPPORT_VAL,
                                      iterations=ITERATIONS)

In [13]:
test_sampler = PrototypicalBatchSampler(labels=test_dataset.class_ids,
                                        data=test_dataset.data_id,
                                      classes_per_it=CLASSES_PER_IT_VAL,
                                      num_samples= NUM_QUERY_VAL+NUM_SUPPORT_VAL,
                                      iterations=ITERATIONS)

In [14]:
train_dataloader = DataLoader(train_dataset, batch_sampler=train_sampler)
val_dataloader = DataLoader(val_dataset, batch_sampler=val_sampler)
test_dataloader = DataLoader(test_dataset, batch_sampler=test_sampler)

### Creating model for googleNet

In [15]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 64)

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [17]:
# initialising optimizer
optim = torch.optim.Adam(params=model.parameters(),
                            lr=LEARNING_RATE)

In [18]:
# initialising learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optim, step_size=LR_STEP, gamma=LR_GAMMA)

### Training the model

In [19]:
trainer = PrototypicalTrainer(tr_dataloader=train_dataloader, 
                              model=model, 
                              optimizer=optim,
                             lr_scheduler=lr_scheduler,
                             device=device,
                             model_path=model_path,
                             iterations=ITERATIONS,
                             n_support=NUM_SUPPORT_TRAINING,
                             n_support_val=NUM_SUPPORT_VAL,
                             val_dataloader=val_dataloader)

In [20]:
res = trainer.train(epochs=EPOCHS)

=== Epoch: 0 ===


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:46<00:00,  2.15it/s]


Avg Train Loss: 1.8133010125160218, Avg Train Acc: 0.2983999998867512
Avg Val Loss: 0.3779999985545874, Avg Val Acc: 1.4436758756637573 (Best)
=== Epoch: 1 ===


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:39<00:00,  2.50it/s]


Avg Train Loss: 1.5310534298419953, Avg Train Acc: 0.3211999984830618
Avg Val Loss: 0.4187999991327524, Avg Val Acc: 1.377099266052246 (Best: 1.4436758756637573)
=== Epoch: 2 ===


...

Avg Train Loss: 1.3469995456933974, Avg Train Acc: 0.44319999903440477
Avg Val Loss: 0.4944000010192394, Avg Val Acc: 1.2087394958734512 (Best: 1.4436758756637573)
=== Epoch: 8 ===


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:42<00:00,  2.34it/s]


Avg Train Loss: 1.3777002549171449, Avg Train Acc: 0.42519999995827673
Avg Val Loss: 0.5559999991953373, Avg Val Acc: 1.095800284743309 (Best: 1.4436758756637573)
=== Epoch: 9 ===


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:42<00:00,  2.33it/s]


Avg Train Loss: 1.3232332217693328, Avg Train Acc: 0.4500000013411045
Avg Val Loss: 0.5228000037372112, Avg Val Acc: 1.1669599622488023 (Best: 1.4436758756637573)


In [21]:
res = trainer.test(test_dataloader)

Test Acc: 0.5174800005555152
